In [167]:
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings("ignore")

In [168]:
# Load dictionarypb.csv and datanullpb.csv separeted by ;
df_dict = pd.read_csv("dictionarypb.csv", sep=";")
df_null = pd.read_csv("datanullpb.csv", sep=";")

In [169]:
# ROW COUNT 
row_count = df_null.shape[0]
print(f"Row count: {row_count}")

Row count: 51999


In [170]:
def validar_y_extraer_u_estilo(itemname):
    try:
        u_estilo = itemname.split('/')[0]

        if len(u_estilo) >= 2:
            primer_caracter = u_estilo[0]
            segundo_caracter = u_estilo[1]
            es_valido = primer_caracter.isalpha() and segundo_caracter.isdigit()
        else:
            es_valido = False
            primer_caracter = None
            segundo_caracter = None

        return pd.Series([u_estilo, primer_caracter, segundo_caracter, es_valido])
    except:
        return pd.Series([None, None, None, False])

# Aplicar al DataFrame
df_null[['u_estilo', 'letra', 'numero', 'es_valido']] = df_null['ItemName'].apply(validar_y_extraer_u_estilo)

# Dividir DataFrames
df_validos = df_null[df_null['es_valido']].copy()
df_invalidos = df_null[~df_null['es_valido']].copy()


# Drop es_valido column
df_validos.drop(columns=['es_valido'], inplace=True)
df_invalidos.drop(columns=['es_valido'], inplace=True)

# DROP letra and numero columns
df_validos.drop(columns=['letra', 'numero'], inplace=True)
df_invalidos.drop(columns=['letra', 'numero'], inplace=True)



In [171]:
df_validos['u_descripcion'] = df_validos['ItemName'].str.split('/').str[1]


In [172]:
df_validos['u_descrip_color'] = df_validos['ItemName'].str.split('/').str[3]

In [173]:
# Crear una versión temporal del df_dict con nombres únicos para evitar conflictos
df_dict_temp = df_dict[['ItemCode', 'Empresa', 'u_genero', 'u_prenda', 'u_subprenda','u_temporalidad']].rename(
    columns={
        'u_genero': 'u_genero_dict',
        'u_prenda': 'u_prenda_dict',
        'u_subprenda': 'u_subprenda_dict'
        ,'u_temporalidad': 'u_temporalidad_dict'
    }
)

# Merge sin crear columnas con sufijos
df_validos = df_validos.merge(df_dict_temp, on=['ItemCode', 'Empresa'], how='left')

# Sobrescribir columnas en df_validos con los valores de df_dict solo si no son nulos
for col in ['u_genero', 'u_prenda', 'u_subprenda','u_temporalidad']:
    col_dict = f"{col}_dict"
    df_validos[col] = df_validos[col_dict].combine_first(df_validos[col])
    df_validos.drop(columns=[col_dict], inplace=True)


In [174]:
df_validos.head()

,ItemName,ItemCode,Empresa,createdate,updatedate,u_descrip_color,u_estilo,u_genero,u_cod_color,u_prenda,u_subprenda,u_sellitem,u_descripcion,u_temporalidad,BarCode
0,B6U579T1PC/CAMISETA FANTASÍA PARA CABALLERO/S/...,31300650232748,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,WHITE,B6U579T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,6.29E+11
1,B6U579T1PC/CAMISETA FANTASÍA PARA CABALLERO/M/...,31300650232848,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,WHITE,B6U579T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,6.29E+11
2,B6U579T1PC/CAMISETA FANTASÍA PARA CABALLERO/L/...,31300650232948,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,WHITE,B6U579T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,6.29E+11
3,B6U579T1PC/CAMISETA FANTASÍA PARA CABALLERO/XL...,31300650233048,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,WHITE,B6U579T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,6.29E+11
4,B6U579T1PC/CAMISETA FANTASÍA PARA CABALLERO/XS...,31300650532648,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,MOJAVE ORANGE,B6U579T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,6.29E+11


In [175]:
df_invalidos.head()

,ItemName,ItemCode,Empresa,createdate,updatedate,u_descrip_color,u_estilo,u_genero,u_cod_color,u_prenda,u_subprenda,u_sellitem,u_descripcion,u_temporalidad,BarCode
4,CAMISETA FANTASÍA PARA CABALLERO WHITE XXL,31300650233148,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,NaN,CAMISETA FANTASÍA PARA CABALLERO WHITE XXL,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
10,CAMISETA FANTASÍA PARA CABALLERO NARANJA XXL,31300650533148,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,NaN,CAMISETA FANTASÍA PARA CABALLERO NARANJA XXL,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
15,CAMISETA FANTASÍA PARA CABALLERO BLACK XXL,31300651013148,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,NaN,CAMISETA FANTASÍA PARA CABALLERO BLACK XXL,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
67,POLO FANTASÍA PARA CABALLERO NEGRO/BLANCO 3XL,31300799554629,SBO_GT_NEWERA,10/04/2023 0:00,13/06/2024 0:00,NaN,POLO FANTASÍA PARA CABALLERO NEGRO,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
76,CAMISETA FANTASÍA PARA CABALLERO BLACK XXL,31300766013148,SBO_GT_NEWERA,06/10/2022 0:00,13/06/2024 0:00,NaN,CAMISETA FANTASÍA PARA CABALLERO BLACK XXL,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN


In [176]:
# quiero que la columna u_estilo este vacia
df_invalidos['u_estilo'] = np.nan


In [177]:
# Renombrar temporalmente columnas de df_dict para evitar duplicados
df_dict_temp = df_dict[['ItemCode', 'Empresa', 'u_genero', 'u_prenda', 'u_subprenda', 'u_temporalidad','u_estilo','u_descripcion','u_descrip_color']].rename(
    columns={
        'u_genero': 'u_genero_dict',
        'u_prenda': 'u_prenda_dict',
        'u_subprenda': 'u_subprenda_dict',
        'u_temporalidad': 'u_temporalidad_dict'
        ,'u_estilo': 'u_estilo_dict'
        ,'u_descripcion': 'u_descripcion_dict'
        ,'u_descrip_color': 'u_descrip_color_dict'
    }
)

# Hacer el merge sin crear columnas duplicadas
df_invalidos = df_invalidos.merge(df_dict_temp, on=['ItemCode', 'Empresa'], how='left')

# Sobrescribir columnas en df_invalidos con valores del dict
for col in ['u_genero', 'u_prenda', 'u_subprenda', 'u_temporalidad', 'u_estilo','u_descripcion','u_descrip_color']:
    col_dict = f"{col}_dict"
    df_invalidos[col] = df_invalidos[col_dict].combine_first(df_invalidos[col])
    df_invalidos.drop(columns=[col_dict], inplace=True)

# Contar valores nulos y no nulos en columnas clave
columnas = ['u_estilo', 'u_genero', 'u_prenda', 'u_subprenda', 'u_temporalidad']



# Ver los primeros registros
df_invalidos.head()



,ItemName,ItemCode,Empresa,createdate,updatedate,u_descrip_color,u_estilo,u_genero,u_cod_color,u_prenda,u_subprenda,u_sellitem,u_descripcion,u_temporalidad,BarCode
0,CAMISETA FANTASÍA PARA CABALLERO WHITE XXL,31300650233148,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,WHITE,B6U584T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,NaN
1,CAMISETA FANTASÍA PARA CABALLERO NARANJA XXL,31300650533148,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,ORANGE,B6U584T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,NaN
2,CAMISETA FANTASÍA PARA CABALLERO BLACK XXL,31300651013148,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,BLACK,B6U584T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,NaN
3,POLO FANTASÍA PARA CABALLERO NEGRO/BLANCO 3XL,31300799554629,SBO_GT_NEWERA,10/04/2023 0:00,13/06/2024 0:00,BLACK,B6K136U1PC,MENS,NaN,FASHION POLOS,POLOS,1,POLO FANTASÍA PARA CABALLERO,FW 22,NaN
4,CAMISETA FANTASÍA PARA CABALLERO BLACK XXL,31300766013148,SBO_GT_NEWERA,06/10/2022 0:00,13/06/2024 0:00,BLACK,B6U944U1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,FW 22,NaN


In [178]:
# concatenar los dos dataframes
df_final = pd.concat([df_validos, df_invalidos], ignore_index=True)
df_final.head()

,ItemName,ItemCode,Empresa,createdate,updatedate,u_descrip_color,u_estilo,u_genero,u_cod_color,u_prenda,u_subprenda,u_sellitem,u_descripcion,u_temporalidad,BarCode
0,B6U579T1PC/CAMISETA FANTASÍA PARA CABALLERO/S/...,31300650232748,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,WHITE,B6U579T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,6.29E+11
1,B6U579T1PC/CAMISETA FANTASÍA PARA CABALLERO/M/...,31300650232848,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,WHITE,B6U579T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,6.29E+11
2,B6U579T1PC/CAMISETA FANTASÍA PARA CABALLERO/L/...,31300650232948,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,WHITE,B6U579T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,6.29E+11
3,B6U579T1PC/CAMISETA FANTASÍA PARA CABALLERO/XL...,31300650233048,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,WHITE,B6U579T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,6.29E+11
4,B6U579T1PC/CAMISETA FANTASÍA PARA CABALLERO/XS...,31300650532648,SBO_GT_NEWERA,29/07/2022 0:00,13/06/2024 0:00,MOJAVE ORANGE,B6U579T1PC,MENS,NaN,GRAPHIC TEES,TEES,1,CAMISETA FANTASÍA PARA CABALLERO,SU 22,6.29E+11


In [179]:
# ROW COUNT
row_count = df_final.shape[0]
print(f"Total rows in df_final: {row_count}")

Total rows in df_final: 51999
